In [2]:
import numpy as np
import pandas as pd
import pyodbc

# if you have less than 10 columns, just include "Extra Columns" for the unused ones otherwise you will break the SQL.
# any column before ColumnUPC doesn't really matter
# remember that these columns must match what's in the workbook exactly
Column1 = 'ASIN'
Column2 = 'VendorCode'
ColumnUPC = 'UPC'
Column4 = 'April'
Column5 = 'May'
Column6 = 'June'
Column7 = 'July'
Column8 = 'August'
Column9 = 'September'
Column10 = 'October'


dataset = pd.read_excel(r'P70 Forecast_KEECS.xlsx', sheet_name='Monthly forecast')
df = pd.DataFrame(dataset)
# The column number must be accurate otherwise it will result in an error
cols = [1,2,3,4,5,6,7,8,9,10]
upc = df[df.columns[cols]]
# Remember you need to be on the VPN for this to work
conn = pyodbc.connect('Driver={Sql Server};'
                      'Server=PDX-SQL-03;'
                      'Database=KEEN_Analytics;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

temp_creator = '''CREATE TABLE #P70_UPC ('''+str(ColumnUPC)+''' bigint, ''' +str(Column4)+'''  
int, ''' +str(Column5)+'''  int, ''' +str(Column6)+'''  int, ''' +str(Column7)+'''  
int, ''' +str(Column8)+'''  int, ''' +str(Column9)+'''  int, ''' +str(Column10)+'''  int);''' 
cursor.execute(temp_creator)
df_insert = upc
df_json = df_insert.to_json(orient='records', double_precision=0)

load_df = '''
 insert into #P70_UPC('''+str(ColumnUPC)+''', '''+str(Column4)+''', '''+str(Column5)+''', '''+str(Column6)+'''
 , '''+str(Column7)+''', '''+str(Column8)+''', '''+str(Column9)+''', '''+str(Column10)+''')
 select '''+str(ColumnUPC)+''', '''+str(Column4)+''', '''+str(Column5)+''', '''+str(Column6)+'''
 , '''+str(Column7)+''', '''+str(Column8)+''', '''+str(Column9)+''', '''+str(Column10)+'''
 from openjson(?)
 with 
 (
   '''+str(ColumnUPC)+''' bigint '$.'''+str(ColumnUPC)+'''',
   '''+str(Column4)+''' int '$.'''+str(Column4)+'''',
   '''+str(Column5)+''' int '$.'''+str(Column5)+'''',
   '''+str(Column6)+''' int '$.'''+str(Column6)+'''',
   '''+str(Column7)+''' int '$.'''+str(Column7)+'''',
   '''+str(Column8)+''' int '$.'''+str(Column8)+'''',
   '''+str(Column9)+''' int '$.'''+str(Column9)+'''',
   '''+str(Column10)+''' int '$.'''+str(Column10)+''''
 );
 '''
cursor.execute(load_df,df_json)
read_query = '''SELECT * FROM #P70_UPC'''
df_back = pd.read_sql(read_query,conn)
read_query = '''WITH [Aggregate] AS (
                SELECT DISTINCT M.Material
                , M.MaterialDescription
                ,CASE WHEN M.Material IN 
                    (SELECT SAPMaterialNo FROM vwCentricLineList WHERE Season = 'Fall 2022' ) 
                    THEN 'INLINE'ELSE 'DISCO' END AS Inline_Stat
                    , P.'''+str(ColumnUPC)+'''
                    , '''+str(Column4)+'''
                    , '''+str(Column5)+'''
                    , '''+str(Column6)+'''
                    , '''+str(Column7)+'''
                    , '''+str(Column8)+'''
                    , '''+str(Column9)+'''
                    , '''+str(Column10)+'''
                    FROM vwUPC U
                INNER JOIN #P70_UPC P
                ON U.UPC = P.'''+str(ColumnUPC)+'''
                INNER JOIN vwKA_Material M
                ON U.Material = M. Material )
                
                SELECT 
                CAST(Material as int) AS Material
                , MaterialDescription
                , Inline_Stat
                , SUM('''+str(Column4)+''') AS '''+str(Column4)+'''
                , SUM('''+str(Column5)+''') AS '''+str(Column5)+'''
                , SUM('''+str(Column6)+''') AS '''+str(Column6)+'''
                , SUM('''+str(Column7)+''') AS '''+str(Column7)+'''
                , SUM('''+str(Column8)+''') AS '''+str(Column8)+'''
                , SUM('''+str(Column9)+''') AS '''+str(Column9)+'''
                , SUM('''+str(Column10)+''') AS '''+str(Column10)+'''
                FROM [Aggregate] A
               
                GROUP BY Material, MaterialDescription, Inline_Stat
                
                ORDER BY '''+str(Column4)+''' DESC
               '''

df_back = pd.read_sql(read_query,conn)

df_back.to_excel(r'P70_Output.xlsx', index = False)